In [49]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Input, Dense, Flatten, Activation, Lambda
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Reshape
from tensorflow.keras.models import Model, Sequential
import numpy as np
import pandas as pd
import tensorflow as tf
import logging
import os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [50]:
def is_nan(x):
    return x is np.nan or x != x

def RemoveNan(file):
    data_nan = pd.read_csv('./data/' + file + '.csv')
    data_nan.info()
    category = data_nan['NewCategory']
    for id in np.arange(len(data_nan)):
        if is_nan(category[id]):
            data_nan = data_nan.drop(id, axis=0)
            # print('remove id: ', id)
    print('drop all lines with nan for ', file)
    data_nan.to_csv('./data/' + file + '-FullLabel.csv')

In [51]:
RemoveNan('more_label')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10397 entries, 0 to 10396
Data columns (total 17 columns):
Unnamed: 0              10397 non-null int64
url                     10397 non-null object
categories              799 non-null object
is entity               2589 non-null float64
html                    10397 non-null object
render                  10397 non-null object
media_introduction      10397 non-null float64
others                  10397 non-null float64
location                10397 non-null float64
social_media_profile    10397 non-null float64
encyclopedia            10397 non-null float64
qa_forum                10397 non-null float64
shopping_item           10397 non-null float64
list                    10397 non-null float64
media_player            10397 non-null float64
article                 10397 non-null float64
NewCategory             4927 non-null object
dtypes: float64(11), int64(1), object(5)
memory usage: 1.3+ MB
drop all lines with nan for  more_label


In [52]:
data = pd.read_csv('./data/more_label-FullLabel.csv')
html_name = data['html']

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4927 entries, 0 to 4926
Data columns (total 18 columns):
Unnamed: 0              4927 non-null int64
Unnamed: 0.1            4927 non-null int64
url                     4927 non-null object
categories              799 non-null object
is entity               1722 non-null float64
html                    4927 non-null object
render                  4927 non-null object
media_introduction      4927 non-null float64
others                  4927 non-null float64
location                4927 non-null float64
social_media_profile    4927 non-null float64
encyclopedia            4927 non-null float64
qa_forum                4927 non-null float64
shopping_item           4927 non-null float64
list                    4927 non-null float64
media_player            4927 non-null float64
article                 4927 non-null float64
NewCategory             4927 non-null object
dtypes: float64(11), int64(2), object(5)
memory usage: 693.0+ KB


In [53]:
# 下面是按照listdir的顺序load vectors，实际应该要按数据对应的html文件的标号来load
maxlen = 300 # sequence 取到的长度，用于下面的padding
input_dir = '/home/yujie6cs/vectors/'
count = 0
x = []

import re
pattern = re.compile(r'\d+')

for id in np.arange(len(html_name)):
    count += 1
    file_name = 'page' + pattern.findall(html_name[id])[0] + '.npy'
    this_x = np.load(os.path.join(input_dir, file_name))
    this_x = this_x.tolist()
    x.append(this_x)
    if count % 1000 == 0:
        logging.info('{} has loaded...'.format(count))
print('load {} vectors done'.format(count))
x = sequence.pad_sequences(x, maxlen=maxlen)

2020-01-16 03:02:29,028 : INFO : 1000 has loaded...
2020-01-16 03:02:29,386 : INFO : 2000 has loaded...
2020-01-16 03:02:29,744 : INFO : 3000 has loaded...
2020-01-16 03:02:30,087 : INFO : 4000 has loaded...


load 4927 vectors done


In [63]:
cate = ['media_introduction', 'others', 'location', 'social_media_profile', 'encyclopedia', \
            'qa_forum', 'shopping_item', 'list', 'media_player', 'article']
y_train = data[cate]

print(x.shape, y_train.shape)

(4927, 300) (4927, 10)


In [83]:
from tensorflow.keras import regularizers

def GetModel(input_shape=(300,)):
    input_vec = Input(shape=input_shape)
    layer1 = Dense(500, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_vec)
    layer2 = Dense(250, activation='relu')(layer1)
    layer3 = Dense(125, activation='relu')(layer2)
    output = Dense(10, activation='softmax')(layer3)
    
    
    # Output Shape: 28x28x1
    clf = Model(input_vec, output)
    clf.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss='categorical_crossentropy',
              metrics=['acc']) # may add f1_score later
    clf.summary()
    return clf

html_clf = GetModel()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 300)]             0         
_________________________________________________________________
dense_30 (Dense)             (None, 500)               150500    
_________________________________________________________________
dense_31 (Dense)             (None, 250)               125250    
_________________________________________________________________
dense_32 (Dense)             (None, 125)               31375     
_________________________________________________________________
dense_33 (Dense)             (None, 10)                1260      
Total params: 308,385
Trainable params: 308,385
Non-trainable params: 0
_________________________________________________________________


In [85]:
html_clf.fit(x, y_train, batch_size=512, epochs=200, verbose=1,
                        shuffle=True, validation_split=0.20)

Train on 3941 samples, validate on 986 samples
Epoch 1/200
3941/3941 [==============================] - 0s 15us/sample - loss: 0.9672 - acc: 0.8774 - val_loss: 0.9747 - val_acc: 0.8641
Epoch 2/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.9638 - acc: 0.8754 - val_loss: 0.9786 - val_acc: 0.8611
Epoch 3/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.9602 - acc: 0.8815 - val_loss: 0.9796 - val_acc: 0.8499
Epoch 4/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.9572 - acc: 0.8785 - val_loss: 0.9813 - val_acc: 0.8621
Epoch 5/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.9535 - acc: 0.8812 - val_loss: 0.9710 - val_acc: 0.8692
Epoch 6/200
3941/3941 [==============================] - 0s 14us/sample - loss: 0.9510 - acc: 0.8843 - val_loss: 0.9881 - val_acc: 0.8540
Epoch 7/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.9473 - acc: 0.8856 - val_loss: 0.9830 - val

3941/3941 [==============================] - 0s 14us/sample - loss: 0.8067 - acc: 0.9411 - val_loss: 1.0289 - val_acc: 0.7982
Epoch 60/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.8049 - acc: 0.9447 - val_loss: 1.0124 - val_acc: 0.7799
Epoch 61/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.8031 - acc: 0.9432 - val_loss: 1.0341 - val_acc: 0.7941
Epoch 62/200
3941/3941 [==============================] - 0s 14us/sample - loss: 0.8011 - acc: 0.9432 - val_loss: 1.0167 - val_acc: 0.7921
Epoch 63/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.8000 - acc: 0.9454 - val_loss: 1.0104 - val_acc: 0.7901
Epoch 64/200
3941/3941 [==============================] - 0s 14us/sample - loss: 0.7977 - acc: 0.9447 - val_loss: 1.0347 - val_acc: 0.7972
Epoch 65/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7951 - acc: 0.9452 - val_loss: 1.0262 - val_acc: 0.8022
Epoch 66/200
3941/3941 [================

Epoch 118/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7400 - acc: 0.9589 - val_loss: 1.0996 - val_acc: 0.7657
Epoch 119/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7381 - acc: 0.9589 - val_loss: 1.0894 - val_acc: 0.7525
Epoch 120/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7408 - acc: 0.9581 - val_loss: 1.1170 - val_acc: 0.7819
Epoch 121/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7390 - acc: 0.9581 - val_loss: 1.1447 - val_acc: 0.7434
Epoch 122/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7382 - acc: 0.9579 - val_loss: 1.1375 - val_acc: 0.7799
Epoch 123/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7398 - acc: 0.9584 - val_loss: 1.1644 - val_acc: 0.7181
Epoch 124/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7391 - acc: 0.9584 - val_loss: 1.1487 - val_acc: 0.7921
Epoch 125/200
3941/3

3941/3941 [==============================] - 0s 13us/sample - loss: 0.7593 - acc: 0.9457 - val_loss: 1.2256 - val_acc: 0.7383
Epoch 177/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7569 - acc: 0.9480 - val_loss: 1.2516 - val_acc: 0.7363
Epoch 178/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7489 - acc: 0.9543 - val_loss: 1.2776 - val_acc: 0.7414
Epoch 179/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7498 - acc: 0.9531 - val_loss: 1.3122 - val_acc: 0.7606
Epoch 180/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7512 - acc: 0.9500 - val_loss: 1.2828 - val_acc: 0.7394
Epoch 181/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7535 - acc: 0.9531 - val_loss: 1.3581 - val_acc: 0.7140
Epoch 182/200
3941/3941 [==============================] - 0s 13us/sample - loss: 0.7547 - acc: 0.9531 - val_loss: 1.3553 - val_acc: 0.7039
Epoch 183/200
3941/3941 [=========